In [56]:
import pandas as pd 
import glob
from bs4 import BeautifulSoup

In [57]:
def store_data(check1,master_dict,tags,soup,email,user_df_list,keys,prin_pi,pal_count,usernames):
        #store palmetto user count 
        if pal_count > 1: 
            palm_user = ','.join(usernames)
            master_dict[keys[5]].append(palm_user)
        elif pal_count == 1:
            try:
                master_dict[keys[5]].append(usernames[0])
            except IndexError:
                master_dict[keys[5]].append('Error')
                pass
        elif pal_count == 0:
            master_dict[keys[5]].append('Not Found')
        #store ID
        awardID = soup.find(tags[0]).string
        master_dict[keys[0]].append(awardID)
        
        #Store Date 
        date = soup.find(tags[1]).string
        master_dict[keys[1]].append(date)
        
        #store cu count 
        master_dict[keys[2]].append(check1) 
        key = [x.string for x in soup.find_all(tags[2])]
        values = [x.string for x in soup.find_all(tags[3])]
        role_dict = dict(zip(key,values))
        #store role check result
        mail = ''
        try:
            for k,v in role_dict.items():
                if v == prin_pi:
                    user = k.split('@')[0]
                    mail = k
            if email in mail:
                if user in user_df_list: 
                    master_dict[keys[4]].append("Yes")
                else: 
                    master_dict[keys[4]].append("No") 
            else:
                master_dict[keys[4]].append("No")
        except KeyError:
            master_dict[keys[4]].append("Error")
            pass    

In [58]:
%%time 

prin_pi = 'Principal Investigator'
email = 'clemson.edu'
tagl = len(tags)
check = 0
pal_count = 0 
usernames = []

tags = [
       'AwardID',
       'AwardEffectiveDate',
       'EmailAddress',
       'RoleCode'
       ]

master_dict = {
               'AwardID':[],
               'Date': [],
               'CU_Email_Count':[],
               'Palmetto_User_Count':[],
               'Principal_Investigator':[],
               'Palmetto_Username':[]
              }
keys = [
        'AwardID',
        'Date',
        'CU_Email_Count',
        'Palmetto_User_Count',
        'Principal_Investigator',
        'Palmetto_Username'
        ]

#csv converted to list 
df = pd.read_csv('../Research-Trend/palmetto/palmetto_data/Users.csv') #type = dataframe
user_df_Series = df['LocalUserId'] #type = Series, palmetto user ids 
user_df_list = list(user_df_Series)
#list of awards 
awards = glob.glob('20*/*.xml')
#parse awards 
for docs in awards:
    #open award
    with open(docs) as award_doc: 
        xml = award_doc.read()
        soup = BeautifulSoup(xml,'xml')
    try: 
        for emails in soup.find_all(tags[2]):
                at = emails.string #email address
                us = emails.string.split('@')[0] #username
                if email in at: 
                    check+=1
                    if us in user_df_list:
                        usernames.append(us)
                        pal_count+=1 #number of palmetto users found 
    except AttributeError:
        check = 0 
        continue
    if check > 0:
        check1 = check 
        check = 0 
        #store palmetto user count  
        master_dict[keys[3]].append(pal_count)
        #store other variables 
        store_data(check1,master_dict,tags,soup,email,user_df_list,keys,
                   prin_pi,pal_count,usernames)
        #reset values
        check1 = 0 
        pal_count = 0
        usernames = []

CPU times: user 10min 14s, sys: 7.92 s, total: 10min 22s
Wall time: 11min 2s


In [59]:
master_df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in master_dict.items()]))
m_df = master_df[['AwardID','Date','CU_Email_Count','Palmetto_User_Count',
                   'Principal_Investigator','Palmetto_Username']]
m_df


,AwardID,Date,CU_Email_Count,Palmetto_User_Count,Principal_Investigator,Palmetto_Username
0,0700508,05/15/2007,1,0,No,Not Found
1,0701187,09/01/2007,1,0,No,Not Found
2,0703042,08/29/2006,1,0,No,Not Found
3,0703061,06/15/2007,3,1,No,mkurz
4,0703117,09/01/2007,2,0,No,Not Found
5,0703547,08/01/2007,3,1,No,kevja
6,0706426,06/01/2007,1,0,No,Not Found
7,0708899,09/01/2007,1,1,Yes,sbritt
8,0710797,11/01/2006,1,0,No,Not Found
9,0722313,08/01/2007,5,1,No,geist


In [60]:
check = m_df.loc[m_df['Palmetto_User_Count'] > 0]
check

,AwardID,Date,CU_Email_Count,Palmetto_User_Count,Principal_Investigator,Palmetto_Username
3,0703061,06/15/2007,3,1,No,mkurz
5,0703547,08/01/2007,3,1,No,kevja
7,0708899,09/01/2007,1,1,Yes,sbritt
9,0722313,08/01/2007,5,1,No,geist
10,0722625,09/01/2007,1,1,Yes,aapon
12,0726023,09/01/2007,1,1,No,sebgoa
13,0728035,09/01/2007,2,1,No,kwang
15,0728868,09/01/2007,2,1,Yes,calkin
17,0733125,08/01/2007,1,1,Yes,mason
20,0733944,08/15/2007,1,1,Yes,hubing


In [61]:
check2 = check.loc[m_df['Principal_Investigator'] == 'Yes']
check2

,AwardID,Date,CU_Email_Count,Palmetto_User_Count,Principal_Investigator,Palmetto_Username
7,0708899,09/01/2007,1,1,Yes,sbritt
10,0722625,09/01/2007,1,1,Yes,aapon
15,0728868,09/01/2007,2,1,Yes,calkin
17,0733125,08/01/2007,1,1,Yes,mason
20,0733944,08/15/2007,1,1,Yes,hubing
35,0800474,05/01/2008,1,1,Yes,gli
51,0820345,02/15/2009,1,1,Yes,ffeltus
55,0825468,10/01/2008,1,1,Yes,bcampb7
69,0838639,02/15/2009,1,1,Yes,dhouse
73,0845593,02/15/2009,1,1,Yes,bcdean
